In [ ]:
import socket

# making a server socket(AF_INET:  tells us what type of address our socket will be able to interact with; in this case a
# hostname and a port number ,  SOCK_STREAM: this means that we
# use the TCP protocol for our communication)
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# SO_REUSEADDR flag to 1. This will allow us to reuse
# the port number after we stop and restart the application
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

address = ('127.0.0.1', 8000)
server_socket.bind(((address[0]),address[1]))

connections = []
server_socket.listen()
try:
    while True:
        connection, client_address = server_socket.accept()
        print(f'I got a connection from {client_address}!')
        connections.append(connection)

        for connection in connections:
            buffer = b''
            while buffer[-2:] != b'\r\n':
                data = connection.recv(2)
                if not data:
                    break
                else:
                    print(f'I got data : {data}')
                    buffer = buffer + data

            print(f"All the data is: {buffer}")
            connection.sendall(buffer)
finally:
    server_socket.close()

Here we have non-blocking socket and will be raised immidiatly. by that socket simply says i have nothing to work wih call me later and provide me data.

In [ ]:
import socket

# making a server socket(AF_INET:  tells us what type of address our socket will be able to interact with; in this case a
# hostname and a port number ,  SOCK_STREAM: this means that we
# use the TCP protocol for our communication)
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# SO_REUSEADDR flag to 1. This will allow us to reuse
# the port number after we stop and restart the application
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

address = ('127.0.0.1', 8000)
server_socket.bind(((address[0]),address[1]))
server_socket.setblocking(False)
connections = []
server_socket.listen()
try:
    while True:
        connection, client_address = server_socket.accept()
        print(f'I got a connection from {client_address}!')
        connections.append(connection)
        connection.setblocking(False)

        for connection in connections:
            buffer = b''
            while buffer[-2:] != b'\r\n':
                data = connection.recv(2)
                if not data:
                    break
                else:
                    print(f'I got data : {data}')
                    buffer = buffer + data

            print(f"All the data is: {buffer}")
            connection.sendall(buffer)
finally:
    server_socket.close()

The solution is to wrap them into a try/except and constantly iterate over and looking for a connection and data otherwise we just ignore the exception and keep iterating.

In [ ]:
import socket

# making a server socket(AF_INET:  tells us what type of address our socket will be able to interact with; in this case a
# hostname and a port number ,  SOCK_STREAM: this means that we
# use the TCP protocol for our communication)
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# SO_REUSEADDR flag to 1. This will allow us to reuse
# the port number after we stop and restart the application
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

address = ('127.0.0.1', 8000)
server_socket.bind(((address[0]),address[1]))
server_socket.setblocking(False)
connections = []
server_socket.listen()
try:
    while True:
        try:
            connection, client_address = server_socket.accept()
            print(f'I got a connection from {client_address}!')
            connections.append(connection)
            connection.setblocking(False)
        except BlockingIOError:
            pass
        for connection in connections:
            try:
                buffer = b''
                while buffer[-2:] != b'\r\n':
                    data = connection.recv(2)
                    if not data:
                        break
                    else:
                        print(f'I got data : {data}')
                        buffer = buffer + data

                print(f"All the data is: {buffer}")
                connection.sendall(buffer)
            except BlockingIOError:
                pass

finally:
    server_socket.close()

But this paricular solution is temporary and very resource consuming. instead we can use event notification system of out OS and hand this task over to operating system to constantly search for data of sockets.

In [ ]:
import selectors
import socket
from selectors import SelectorKey
from typing import List, Tuple
selector = selectors.DefaultSelector()
server_socket = socket.socket()
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
server_address = ('127.0.0.1', 8000)
server_socket.setblocking(False)
server_socket.bind(server_address)
server_socket.listen()
selector.register(server_socket, selectors.EVENT_READ)

while True:
    events: List[Tuple[SelectorKey, int]] = selector.select(timeout=1) 
    if len(events) == 0: 
        print('No events, waiting a bit more!')
    for event, _ in events:
        event_socket = event.fileobj 
        if event_socket == server_socket: 
            connection, address = server_socket.accept()
            connection.setblocking(False)
            print(f"I got a connection from {address}")
            selector.register(connection, selectors.EVENT_READ) 

        else:
            data = event_socket.recv(1024) 
            print(f"I got some data: {data}")
            event_socket.send(data)